In [1]:
from bertviz import model_view
from transformers import AutoTokenizer, AutoModel
import pickle

# TODO : torch text should be 0.11.2 // torch = 1.10.2
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import torch

/home/kchen91/anaconda3/envs/former/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = get_tokenizer('basic_english')
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

In [3]:
train_iter = IMDB(split='train')
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: 0 if x=='neg' else 1

In [25]:
sentence = "this is a book on the table"
processed_text = torch.tensor(text_pipeline(sentence), dtype=torch.int64)
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)


In [26]:
processed_text= processed_text.view(1, -1)

In [27]:
model.train(False)
out, att = model(processed_text)

In [31]:
att = att.reshape(1,8,7,7)
att = (att[None,:])

In [32]:
att

tensor([[[[[0.1576, 0.1489, 0.1401, 0.1156, 0.1551, 0.1540, 0.1286],
           [0.1373, 0.1419, 0.1703, 0.1390, 0.1236, 0.1607, 0.1272],
           [0.1450, 0.1382, 0.1527, 0.1355, 0.1268, 0.1598, 0.1420],
           [0.1162, 0.1598, 0.1598, 0.1388, 0.1315, 0.1394, 0.1545],
           [0.1702, 0.1267, 0.1436, 0.1306, 0.1650, 0.1412, 0.1227],
           [0.1540, 0.1462, 0.1437, 0.1394, 0.1513, 0.1493, 0.1160],
           [0.1346, 0.1383, 0.1418, 0.1578, 0.1284, 0.1570, 0.1421]],

          [[0.1473, 0.1440, 0.1706, 0.1506, 0.1274, 0.1268, 0.1333],
           [0.1337, 0.1425, 0.1466, 0.1210, 0.1511, 0.1538, 0.1513],
           [0.1552, 0.1507, 0.1614, 0.1250, 0.1073, 0.1465, 0.1539],
           [0.1290, 0.1353, 0.1768, 0.1327, 0.1440, 0.1346, 0.1475],
           [0.1253, 0.1408, 0.1371, 0.1611, 0.1482, 0.1332, 0.1543],
           [0.1338, 0.1243, 0.1726, 0.1369, 0.1376, 0.1398, 0.1550],
           [0.1358, 0.1219, 0.1581, 0.1411, 0.1381, 0.1420, 0.1630]],

          [[0.1590, 0.1356, 0.

In [30]:
model_view(att, processed_text, sentence)

TypeError: slice indices must be integers or None or have an __index__ method

In [15]:
import former
from former import util
from former.util import d, here

import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

from torchtext import data, datasets, vocab
# from torchtext.legacy import data, datasets, vocab

import numpy as np

from argparse import ArgumentParser, Namespace
from torch.utils.tensorboard import SummaryWriter

import random, tqdm, sys, math, gzip
import os
from copy import deepcopy
from datetime import datetime
# Used for converting between nats and bits
LOG2E = math.log2(math.e)
TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
LABEL = data.Field(sequential=False)
NUM_CLS = 2
MODEL_OUTPUT_PATH = './model_output'



def go(arg):
    """
    Creates and trains a basic transformer for the IMDB sentiment classification task.
    """
    tbw = SummaryWriter(log_dir=arg.tb_dir) # Tensorboard logging

    # load the IMDB data
    if arg.final:
        train, test = datasets.IMDB.splits(TEXT, LABEL)


        TEXT.build_vocab(train, max_size=arg.vocab_size - 2)
        LABEL.build_vocab(train)

        train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=arg.batch_size, device=util.d())
    else:
        tdata, _ = datasets.IMDB.splits(TEXT, LABEL)
        print(f'tdata ======> ',tdata.split())
        train, test = tdata.split(split_ratio=0.8)

        TEXT.build_vocab(train, max_size=arg.vocab_size - 2) # - 2 to make space for <unk> and <pad>
        LABEL.build_vocab(train)

        train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=arg.batch_size, device=util.d())

    print(f'- nr. of training examples {len(train_iter)}')
    print(f'- nr. of {"test" if arg.final else "validation"} examples {len(test_iter)}')

    if arg.max_length < 0:
        mx = max([input.text[0].size(1) for input in train_iter])
        mx = mx * 2
        print(f'- maximum sequence length: {mx}')
    else:
        mx = arg.max_length

    # create the model
    model = former.CTransformer(emb=arg.embedding_size, heads=arg.num_heads, depth=arg.depth, seq_length=mx, num_tokens=arg.vocab_size, num_classes=NUM_CLS, max_pool=arg.max_pool)
    if torch.cuda.is_available():
        model.cuda()

    model = nn.DataParallel(model, device_ids=[0, 1, 2])

    opt = torch.optim.SGD(
        lr=arg.lr, params=model.parameters(), momentum=arg.momentum)

    # opt = torch.optim.Adam(lr=arg.lr, params=model.parameters())
    sch = torch.optim.lr_scheduler.LambdaLR(opt, lambda i: min(i / (arg.lr_warmup / arg.batch_size), 1.0))

    # best output to save model
    best_accuracy = 0
    best_model_state = None
    # training loop
    seen = 0
    for e in range(arg.num_epochs):

        print(f'\n epoch {e}')
        model.train(True)

        for batch in tqdm.tqdm(train_iter):

            opt.zero_grad()

            input = batch.text[0]
            label = batch.label - 1

            if input.size(1) > mx:
                input = input[:, :mx]
            out, att = model(input)
            print()
            loss = F.nll_loss(out, label)

            loss.backward()

            # clip gradients
            # - If the total gradient vector has a length > 1, we clip it back down to 1.
            if arg.gradient_clipping > 0.0:
                nn.utils.clip_grad_norm_(model.parameters(), arg.gradient_clipping)

            opt.step()
            sch.step()

            seen += input.size(0)
            tbw.add_scalar('classification/train-loss', float(loss.item()), seen)

        with torch.no_grad():

            model.train(False)
            tot, cor= 0.0, 0.0

            for batch in test_iter:

                input = batch.text[0]
                label = batch.label - 1

                if input.size(1) > mx:
                    input = input[:, :mx]
                out = model(input).argmax(dim=1)

                tot += float(input.size(0))
                cor += float((label == out).sum().item())

            acc = cor / tot
            print(f'-- {"test" if arg.final else "validation"} accuracy {acc:.3}')
            tbw.add_scalar('classification/test-loss', float(loss.item()), e)
            # if acc > best_accuracy:
            #     best_accuracy, best_model_state = acc, deepcopy(
            #         model.state_dict())

    return model

    # if not os.path.exists(MODEL_OUTPUT_PATH):
    #     os.mkdir(MODEL_OUTPUT_PATH)
    # time_stamp = datetime.now()
    # torch.save(best_model_state, MODEL_OUTPUT_PATH +
    #            f'{time_stamp}_SGD_momentum_{arg.momentum}_acc_{best_accuracy:3f}')



In [16]:

parser = ArgumentParser()


parser.add_argument("-e", "--num-epochs",
                        dest="num_epochs",
                        help="Number of epochs.",
                        default=80, type=int)

parser.add_argument("-b", "--batch-size",
                        dest="batch_size",
                        help="The batch size.",
                        default=4, type=int)

parser.add_argument("-l", "--learn-rate",
                        dest="lr",
                        help="Learning rate",
                        default=0.0001, type=float)

parser.add_argument("-T", "--tb_dir", dest="tb_dir",
                        help="Tensorboard logging directory",
                        default='./runs')

parser.add_argument("-f", "--final", dest="final",
                        help="Whether to run on the real test set (if not included, the validation set is used).",
                        action="store_true")

parser.add_argument("--max-pool", dest="max_pool",
                    help="Use max pooling in the final classification layer.",
                    action="store_true")

parser.add_argument("-E", "--embedding", dest="embedding_size",
                    help="Size of the character embeddings.",
                    default=128, type=int)

parser.add_argument("-V", "--vocab-size", dest="vocab_size",
                    help="Number of words in the vocabulary.",
                
                    default=50_000, type=int)

parser.add_argument("-M", "--max", dest="max_length",
                    help="Max sequence length. Longer sequences are clipped (-1 for no limit).",
                    default=512, type=int)

parser.add_argument("-H", "--heads", dest="num_heads",
                    help="Number of attention heads.",
                    default=8, type=int)

parser.add_argument("-d", "--depth", dest="depth",
                    help="Depth of the network (nr. of self-attention layers)",
                    default=6, type=int)

parser.add_argument("-r", "--random-seed",
                    dest="seed",
                    help="RNG seed. Negative for random",
                    default=1, type=int)

parser.add_argument("--lr-warmup",
                    dest="lr_warmup",
                    help="Learning rate warmup.",
                    default=10_000, type=int)

parser.add_argument("--gradient-clipping",
                    dest="gradient_clipping",
                    help="Gradient clipping.",
                    default=1.0, type=float)

parser.add_argument("--momentum",
                    dest="momentum",
                    help="momentum for SGD",
                    default=0.9, type=float)

# args, unknown = parser.parse_known_args()


args = Namespace(
    num_epochs=80,
    batch_size=4,
    lr=0.0001,
    tb_dir='./runs',
    final=False,
    max_pool=False,
    embedding_size=128,
    vocab_size=50_000,
    max_length=512,
    num_heads=8,
    depth=6,
    seed=1,
    lr_warmup=10_000,
    gradient_clipping=1.0,
    momentum=0.9
)

model = go(arg=args)

aclImdb_v1.tar.gz:   4%|▎         | 3.08M/84.1M [00:00<00:02, 30.8MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:00<00:00, 90.2MB/s]


tdata ======>  (<torchtext.data.dataset.Dataset object at 0x7f17ce3a0450>, <torchtext.data.dataset.Dataset object at 0x7f17ce35c6d0>)
- nr. of training examples 5000
- nr. of validation examples 1250


  0%|          | 0/5000 [00:00<?, ?it/s]

...

 36%|███▌      | 1793/5000 [00:58<01:45, 30.31it/s]

 36%|███▌      | 1801/5000 [00:58<01:41, 31.54it/s]

 36%|███▌      | 1805/5000 [00:59<01:43, 30.97it/s]

KeyboardInterrupt: 

 36%|███▌      | 1805/5000 [01:10<01:43, 30.97it/s]

In [ ]:
if __name__ == "__main__":

    parser = ArgumentParser()

    parser.add_argument("-e", "--num-epochs",
                        dest="num_epochs",
                        help="Number of epochs.",
                        default=80, type=int)

    parser.add_argument("-b", "--batch-size",
                        dest="batch_size",
                        help="The batch size.",
                        default=4, type=int)

    parser.add_argument("-l", "--learn-rate",
                        dest="lr",
                        help="Learning rate",
                        default=0.0001, type=float)

    parser.add_argument("-T", "--tb_dir", dest="tb_dir",
                        help="Tensorboard logging directory",
                        default='./runs')

    parser.add_argument("-f", "--final", dest="final",
                        help="Whether to run on the real test set (if not included, the validation set is used).",
                        action="store_true")

    parser.add_argument("--max-pool", dest="max_pool",
                        help="Use max pooling in the final classification layer.",
                        action="store_true")

    parser.add_argument("-E", "--embedding", dest="embedding_size",
                        help="Size of the character embeddings.",
                        default=128, type=int)

    parser.add_argument("-V", "--vocab-size", dest="vocab_size",
                        help="Number of words in the vocabulary.",
                        default=50_000, type=int)

    parser.add_argument("-M", "--max", dest="max_length",
                        help="Max sequence length. Longer sequences are clipped (-1 for no limit).",
                        default=512, type=int)

    parser.add_argument("-H", "--heads", dest="num_heads",
                        help="Number of attention heads.",
                        default=8, type=int)

    parser.add_argument("-d", "--depth", dest="depth",
                        help="Depth of the network (nr. of self-attention layers)",
                        default=6, type=int)

    parser.add_argument("-r", "--random-seed",
                        dest="seed",
                        help="RNG seed. Negative for random",
                        default=1, type=int)

    parser.add_argument("--lr-warmup",
                        dest="lr_warmup",
                        help="Learning rate warmup.",
                        default=10_000, type=int)

    parser.add_argument("--gradient-clipping",
                        dest="gradient_clipping",
                        help="Gradient clipping.",
                        default=1.0, type=float)

    parser.add_argument("--momentum",
                        dest="momentum",
                        help="momentum for SGD",
                        default=0.9, type=float)

    options = parser.parse_args()

    print('OPTIONS ', options)

    model = go(options)